# Data preparation

In [14]:
import pandas as pd
df = pd.read_csv('S&P_500/AAPL_diff.csv')
df = df.drop(columns=['Date', 'Name'])

In [6]:
#We will use forward chaining KFold split strategy,
#For this sklearn has function called TimeSeriesSplit

In [9]:
from sklearn.model_selection import TimeSeriesSplit
splitter = TimeSeriesSplit(n_splits=10)

In [27]:
for indices_x, indices_y in splitter.split(df):
    print(indices_x.shape, indices_y.shape)

(118,) (114,)
(232,) (114,)
(346,) (114,)
(460,) (114,)
(574,) (114,)
(688,) (114,)
(802,) (114,)
(916,) (114,)
(1030,) (114,)
(1144,) (114,)


# Model definition

In [ ]:
import torch
import torch.nn as nn
import numpy as np

# Define the transformer model
class SimpleTransformer(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_layers):
        super(TransformerModel, self).__init__()
        self.transformer = nn.Transformer(d_model, nhead, num_layers)
        self.fc = nn.Linear(d_model, input_dim)
        
    def forward(self, x):
        x = x.transpose(0,1)
        x = self.transformer(x)
        x = self.fc(x)
        return x.transpose(0,1)

# Create an instance of the model
input_dim = 1
d_model = 128
nhead = 2
num_layers = 3
model = SimpleTransformer(input_dim, d_model, nhead, num_layers)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


# Set the model to training mode
model.train()

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    y_pred = model(X_train)

    # Compute the loss
    loss = criterion(y_pred, X_train)

    # Zero gradients, perform a backward pass, and update the weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')


In [ ]:
def train_kfolds(data, indices_generator, n_folds, d_model, nhead, num_layers):
    for fold_counter in range(n_folds):
        #define model
        model = SimpleTransformer(input_dim, d_model, nhead, num_layers)

        # Define the loss function and optimizer
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        model.train()

        # Train the model
        num_epochs = 100
        for epoch in range(num_epochs):
            # Forward pass
            y_pred = model(X_train)

            # Compute the loss
            loss = criterion(y_pred, X_train)

            # Zero gradients, perform a backward pass, and update the weights
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (epoch+1) % 10 == 0:
                print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')
